# Chapter 4.2

## Case Statements
Case statements provide a cleaner alternative to multiple if-else statements when comparing a single expression against multiple values.

## case Statement

The standard `case` statement performs exact matching including X and Z values.

```systemverilog
case (expression)
    value1: statement1;
    value2: statement2;
    value3, value4: statement3; // Multiple values
    default: default_statement;
endcase
```

### Example 3: ALU Design

In [4]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_4_examples/example_3__alu/"
files = ["alu.sv", "alu_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// alu.sv
module alu(
    input logic [3:0] opcode,
    input logic [7:0] a, b,
    output logic [7:0] result,
    output logic zero
);
    always_comb begin
        case (opcode)
            4'b0000: result = a + b;        // ADD
            4'b0001: result = a - b;        // SUB
            4'b0010: result = a & b;        // AND
            4'b0011: result = a | b;        // OR
            4'b0100: result = a ^ b;        // XOR
            4'b0101: result = ~a;           // NOT
            4'b0110: result = a << 1;       // Shift left
            4'b0111: result = a >> 1;       // Shift right
            default: result = 8'h00;
        endcase
        
        zero = (result == 8'h00);
    end
endmodule
```

```systemverilog
// alu_testbench.sv
module alu_testbench;  // Testbench module
    
    // Testbench signals
    logic [3:0] opcode;
    logic [7:0] a, b;
    logic [7:0] result;
    logic zero;
    
    // Test counters
    integer test_count = 0;
    integer pass_count = 0;
    
    // ALU operation names for display
    string op_names[8] = '{
        "ADD", "SUB", "AND", "OR", "XOR", "NOT", "SHL", "SHR"
    };
    
    // Instantiate design under test
    alu DUT (
        .opcode(opcode),
        .a(a),
        .b(b),
        .result(result),
        .zero(zero)
    );

    // Task to run a test case
    task run_test(
        input [3:0] test_opcode,
        input [7:0] test_a, test_b,
        input [7:0] expected_result,
        input expected_zero,
        input string test_description
    );
        test_count++;
        opcode = test_opcode;
        a = test_a;
        b = test_b;
        #1; // Wait for combinational logic to settle
        
        $display("Test %0d: %s", test_count, test_description);
        if (test_opcode <= 4'b0111) begin
            $display("  Operation: %s (opcode=4'b%04b)", op_names[test_opcode[2:0]], test_opcode);
        end else begin
            $display("  Operation: INVALID (opcode=4'b%04b)", test_opcode);
        end
        $display("  Inputs: a=8'h%02h (%0d), b=8'h%02h (%0d)", a, a, b, b);
        $display("  Result: 8'h%02h (%0d), zero=%b", result, result, zero);
        $display("  Expected: 8'h%02h (%0d), zero=%b", expected_result, expected_result, expected_zero);
        
        if (result == expected_result && zero == expected_zero) begin
            $display("PASS");
            pass_count++;
        end else begin
            $display("FAIL");
            $error("Test %0d failed: Expected result=8'h%02h, zero=%b, got result=8'h%02h, zero=%b", 
                   test_count, expected_result, expected_zero, result, zero);
        end
        $display();
    endtask

    initial begin
        // Dump waves
        $dumpfile("alu_testbench.vcd");       // Specify the VCD file
        $dumpvars(0, alu_testbench);          // Dump all variables in the test module
        
        $display();
        $display("Hello from testbench!");
        $display("Starting ALU Tests");
        $display("==================");
        $display("8-bit ALU with 4-bit opcode");
        $display("Operations: ADD(0), SUB(1), AND(2), OR(3), XOR(4), NOT(5), SHL(6), SHR(7)");
        $display("Zero flag indicates when result equals 0");
        $display();

        // Test 1: ADD operations
        run_test(4'b0000, 8'h0F, 8'h10, 8'h1F, 1'b0, "ADD: 15 + 16 = 31");
        run_test(4'b0000, 8'hFF, 8'h01, 8'h00, 1'b1, "ADD: 255 + 1 = 0 (overflow, zero flag set)");
        run_test(4'b0000, 8'h00, 8'h00, 8'h00, 1'b1, "ADD: 0 + 0 = 0 (zero flag set)");
        run_test(4'b0000, 8'h7F, 8'h7F, 8'hFE, 1'b0, "ADD: 127 + 127 = 254");

        // Test 2: SUB operations
        run_test(4'b0001, 8'h20, 8'h10, 8'h10, 1'b0, "SUB: 32 - 16 = 16");
        run_test(4'b0001, 8'h10, 8'h10, 8'h00, 1'b1, "SUB: 16 - 16 = 0 (zero flag set)");
        run_test(4'b0001, 8'h10, 8'h20, 8'hF0, 1'b0, "SUB: 16 - 32 = -16 (underflow)");
        run_test(4'b0001, 8'hFF, 8'h01, 8'hFE, 1'b0, "SUB: 255 - 1 = 254");

        // Test 3: AND operations
        run_test(4'b0010, 8'hFF, 8'hAA, 8'hAA, 1'b0, "AND: 0xFF & 0xAA = 0xAA");
        run_test(4'b0010, 8'hF0, 8'h0F, 8'h00, 1'b1, "AND: 0xF0 & 0x0F = 0x00 (zero flag set)");
        run_test(4'b0010, 8'hFF, 8'hFF, 8'hFF, 1'b0, "AND: 0xFF & 0xFF = 0xFF");
        run_test(4'b0010, 8'h55, 8'hAA, 8'h00, 1'b1, "AND: 0x55 & 0xAA = 0x00");

        // Test 4: OR operations
        run_test(4'b0011, 8'hF0, 8'h0F, 8'hFF, 1'b0, "OR: 0xF0 | 0x0F = 0xFF");
        run_test(4'b0011, 8'h00, 8'h00, 8'h00, 1'b1, "OR: 0x00 | 0x00 = 0x00 (zero flag set)");
        run_test(4'b0011, 8'h55, 8'hAA, 8'hFF, 1'b0, "OR: 0x55 | 0xAA = 0xFF");
        run_test(4'b0011, 8'h12, 8'h34, 8'h36, 1'b0, "OR: 0x12 | 0x34 = 0x36");

        // Test 5: XOR operations
        run_test(4'b0100, 8'hFF, 8'hFF, 8'h00, 1'b1, "XOR: 0xFF ^ 0xFF = 0x00 (zero flag set)");
        run_test(4'b0100, 8'h55, 8'hAA, 8'hFF, 1'b0, "XOR: 0x55 ^ 0xAA = 0xFF");
        run_test(4'b0100, 8'hF0, 8'h0F, 8'hFF, 1'b0, "XOR: 0xF0 ^ 0x0F = 0xFF");
        run_test(4'b0100, 8'h12, 8'h12, 8'h00, 1'b1, "XOR: 0x12 ^ 0x12 = 0x00");

        // Test 6: NOT operations (b input ignored)
        run_test(4'b0101, 8'hFF, 8'h00, 8'h00, 1'b1, "NOT: ~0xFF = 0x00 (zero flag set)");
        run_test(4'b0101, 8'h00, 8'hFF, 8'hFF, 1'b0, "NOT: ~0x00 = 0xFF");
        run_test(4'b0101, 8'hAA, 8'h00, 8'h55, 1'b0, "NOT: ~0xAA = 0x55");
        run_test(4'b0101, 8'hF0, 8'h00, 8'h0F, 1'b0, "NOT: ~0xF0 = 0x0F");

        // Test 7: Shift Left operations (b input ignored)
        run_test(4'b0110, 8'h01, 8'h00, 8'h02, 1'b0, "SHL: 0x01 << 1 = 0x02");
        run_test(4'b0110, 8'h80, 8'h00, 8'h00, 1'b1, "SHL: 0x80 << 1 = 0x00 (MSB lost, zero flag set)");
        run_test(4'b0110, 8'h55, 8'h00, 8'hAA, 1'b0, "SHL: 0x55 << 1 = 0xAA");
        run_test(4'b0110, 8'h7F, 8'h00, 8'hFE, 1'b0, "SHL: 0x7F << 1 = 0xFE");

        // Test 8: Shift Right operations (b input ignored)
        run_test(4'b0111, 8'h02, 8'h00, 8'h01, 1'b0, "SHR: 0x02 >> 1 = 0x01");
        run_test(4'b0111, 8'h01, 8'h00, 8'h00, 1'b1, "SHR: 0x01 >> 1 = 0x00 (LSB lost, zero flag set)");
        run_test(4'b0111, 8'hAA, 8'h00, 8'h55, 1'b0, "SHR: 0xAA >> 1 = 0x55");
        run_test(4'b0111, 8'hFE, 8'h00, 8'h7F, 1'b0, "SHR: 0xFE >> 1 = 0x7F");

        // Test 9: Invalid opcodes (default case)
        run_test(4'b1000, 8'hFF, 8'hFF, 8'h00, 1'b1, "Invalid opcode 8 -> default result 0x00");
        run_test(4'b1111, 8'hAA, 8'h55, 8'h00, 1'b1, "Invalid opcode 15 -> default result 0x00");

        // Test 10: Edge cases
        run_test(4'b0000, 8'h80, 8'h80, 8'h00, 1'b1, "ADD edge: 0x80 + 0x80 = 0x00 (overflow)");
        run_test(4'b0001, 8'h00, 8'h01, 8'hFF, 1'b0, "SUB edge: 0x00 - 0x01 = 0xFF (underflow)");

        // Summary
        $display("Test Summary:");
        $display("============");
        $display("Total tests: %0d", test_count);
        $display("Passed: %0d", pass_count);
        $display("Failed: %0d", test_count - pass_count);
        
        if (pass_count == test_count) begin
            $display("ALL TESTS PASSED!");
        end else begin
            $display("Some tests failed. Please review.");
        end
        
        $display();
        $display("ALU Testing Complete!");
        $display("====================");
        $display();
        
        $finish;  // End simulation
    end

endmodule
```

Verilator Simulation Output:

Hello from testbench!
Starting ALU Tests
8-bit ALU with 4-bit opcode
Operations: ADD(0), SUB(1), AND(2), OR(3), XOR(4), NOT(5), SHL(6), SHR(7)
Zero flag indicates when result equals 0

Test 1: ADD: 15 + 16 = 31
  Operation: ADD (opcode=4'b0000)
  Inputs: a=8'h0f (15), b=8'h10 (16)
  Result: 8'h1f (31), zero=0
  Expected: 8'h1f (31), zero=0
PASS

Test 2: ADD: 255 + 1 = 0 (overflow, zero flag set)
  Operation: ADD (opcode=4'b0000)
  Inputs: a=8'hff (255), b=8'h01 (1)
  Result: 8'h00 (0), zero=1
  Expected: 8'h00 (0), zero=1
PASS

Test 3: ADD: 0 + 0 = 0 (zero flag set)
  Operation: ADD (opcode=4'b0000)
  Inputs: a=8'h00 (0), b=8'h00 (0)
  Result: 8'h00 (0), zero=1
  Expected: 8'h00 (0), zero=1
PASS

Test 4: ADD: 127 + 127 = 254
  Operation: ADD (opcode=4'b0000)
  Inputs: a=8'h7f (127), b=8'h7f (127)
  Result: 8'hfe (254), zero=0
  Expected: 8'hfe (254), zero=0
PASS

Test 5: SUB: 32 - 16 = 16
  Operation: SUB (opcode=4'b0001)
  Inputs: a=8'h20 (32), b=8'h10 (1

0

## casex Statement

`casex` treats X and Z as don't-care values in both the case expression and case items.

```systemverilog
casex (data)
    4'b1???: // Matches any 4-bit value starting with 1
        result = "starts_with_1";
    4'b?1??: // Matches any 4-bit value with second bit as 1
        result = "second_bit_1";
    default:
        result = "other";
endcase
```

### Example 4: Instruction Decoder

In [5]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_4_examples/example_4__instruction_decoder/"
files = ["instruction_decoder.sv", "instruction_decoder_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// instruction_decoder.sv
module instruction_decoder(
    input logic [7:0] instruction,
    output logic [2:0] op_type
);
    always_comb begin
        /* verilator lint_off CASEX */
        casex (instruction)
            8'b000?????: op_type = 3'b001;  // Load instructions
            8'b001?????: op_type = 3'b010;  // Store instructions
            8'b010?????: op_type = 3'b011;  // Arithmetic
            8'b011?????: op_type = 3'b100;  // Logic
            8'b1???????: op_type = 3'b101;  // Branch
            default:     op_type = 3'b000;  // NOP
        endcase
        /* verilator lint_on CASEX */
    end
endmodule
```

```systemverilog
// instruction_decoder_testbench.sv
module instruction_decoder_testbench;
    // Testbench signals
    logic [7:0] instruction;
    logic [2:0] op_type;
    
    // Instantiate the design under test
    instruction_decoder DUT (
        .instruction(instruction),
        .op_type(op_type)
    );
    
    // Test stimulus
    initial begin
        // Dump waves
        $dumpfile("instruction_decoder_testbench.vcd");
        $dumpvars(0, instruction_decoder_testbench);
        
        $display("Starting Instruction Decoder Test");
        $display("=====================================");
        $display();
        
        // Test Load instructions (000?????)
        instruction = 8'b00000000; #1;
        $display("Instruction: %b, Op Type: %b (Expected: 001 - Load)", instruction, op_type);
        instruction = 8'b00011111; #1;
        $display("Instruction: %b, Op Type: %b (Expected: 001 - Load)", instruction, op_type);
        
        // Test Store instructions (001?????)
        instruction = 8'b00100000; #1;
        $display("Instruction: %b, Op Type: %b (Expected: 010 - Store)", instruction, op_type);
        instruction = 8'b00111111; #1;
        $display("Instruction: %b, Op Type: %b (Expected: 010 - Store)", instruction, op_type);
        
        // Test Arithmetic instructions (010?????)
        instruction = 8'b01000000; #1;
        $display("Instruction: %b, Op Type: %b (Expected: 011 - Arithmetic)", instruction, op_type);
        instruction = 8'b01011111; #1;
        $display("Instruction: %b, Op Type: %b (Expected: 011 - Arithmetic)", instruction, op_type);
        
        // Test Logic instructions (011?????)
        instruction = 8'b01100000; #1;
        $display("Instruction: %b, Op Type: %b (Expected: 100 - Logic)", instruction, op_type);
        instruction = 8'b01111111; #1;
        $display("Instruction: %b, Op Type: %b (Expected: 100 - Logic)", instruction, op_type);
        
        // Test Branch instructions (1???????)
        instruction = 8'b10000000; #1;
        $display("Instruction: %b, Op Type: %b (Expected: 101 - Branch)", instruction, op_type);
        instruction = 8'b11111111; #1;
        $display("Instruction: %b, Op Type: %b (Expected: 101 - Branch)", instruction, op_type);
        instruction = 8'b10101010; #1;
        $display("Instruction: %b, Op Type: %b (Expected: 101 - Branch)", instruction, op_type);
        
        $display();
        $display("Test completed!");
        $display("=====================================");
        
        $finish;
    end

endmodule
```

Verilator Simulation Output:
Starting Instruction Decoder Test

Instruction: 00000000, Op Type: 001 (Expected: 001 - Load)
Instruction: 00011111, Op Type: 001 (Expected: 001 - Load)
Instruction: 00100000, Op Type: 010 (Expected: 010 - Store)
Instruction: 00111111, Op Type: 010 (Expected: 010 - Store)
Instruction: 01000000, Op Type: 011 (Expected: 011 - Arithmetic)
Instruction: 01011111, Op Type: 011 (Expected: 011 - Arithmetic)
Instruction: 01100000, Op Type: 100 (Expected: 100 - Logic)
Instruction: 01111111, Op Type: 100 (Expected: 100 - Logic)
Instruction: 10000000, Op Type: 101 (Expected: 101 - Branch)
Instruction: 11111111, Op Type: 101 (Expected: 101 - Branch)
Instruction: 10101010, Op Type: 101 (Expected: 101 - Branch)

Test completed!
Process finished with return code: 0
Removing Chapter_4_examples/example_4__instruction_decoder/obj_dir directory...
Chapter_4_examples/example_4__instruction_decoder/obj_dir removed successfully.


0

## casez Statement

`casez` treats only Z as don't-care values (more restrictive than casex).

```systemverilog
casez (selector)
    4'b1zzz: output = input1;
    4'bz1zz: output = input2;
    default: output = default_val;
endcase
```

### Case Statement Guidelines
- Always include a `default` case
- Use `casex` for don't-care matching
- Use `casez` when only Z should be treated as don't-care
- Avoid overlapping case items

### unique and priority Modifiers

SystemVerilog provides `unique` and `priority` modifiers to specify the intent and improve synthesis results.

#### unique Modifier

The `unique` modifier indicates that case items are mutually exclusive and exactly one will match.

```systemverilog
unique case (state)
    IDLE:  next_state = START;
    START: next_state = ACTIVE;
    ACTIVE: next_state = DONE;
    DONE:  next_state = IDLE;
endcase
```

#### priority Modifier

The `priority` modifier indicates that case items should be evaluated in order, and at least one will match.

```systemverilog
priority case (1'b1)
    error_flag:     status = ERROR;
    warning_flag:   status = WARNING;
    ready_flag:     status = READY;
    default:        status = IDLE;
endcase
```

### Example 5: Finite State Machine

In [6]:
# | echo: false

from IPython.display import Markdown, display
from verilator_runner import run_docker_compose

files_path = "Chapter_4_examples/example_5__fsm/"
files = ["fsm.sv", "fsm_testbench.sv"]

# Read SystemVerilog code from file
for file in files:
    with open(f"{files_path}/{file}", "r") as source_file:
        sv_code = source_file.read()

    display(Markdown(f"```systemverilog\n{sv_code}\n```"))

run_docker_compose(target_dir=f"{files_path}", strip_lines=True)


```systemverilog
// fsm.sv
typedef enum logic [1:0] {
    IDLE = 2'b00,
    READ = 2'b01,
    WRITE = 2'b10,
    DONE = 2'b11
} state_t;

module fsm(
    input logic clk, rst_n, start, rw,
    output logic busy, done
);
    state_t current_state, next_state;
    
    always_ff @(posedge clk or negedge rst_n) begin
        if (!rst_n)
            current_state <= IDLE;
        else
            current_state <= next_state;
    end
    
    always_comb begin
        unique case (current_state)
            IDLE: begin
                if (start)
                    next_state = rw ? WRITE : READ;
                else
                    next_state = IDLE;
            end
            READ: next_state = DONE;
            WRITE: next_state = DONE;
            DONE: next_state = IDLE;
        endcase
    end
    
    assign busy = (current_state != IDLE);
    assign done = (current_state == DONE);
endmodule
```

```systemverilog
// fsm_testbench.sv
module fsm_testbench;
    // Testbench signals
    logic clk, rst_n, start, rw;
    logic busy, done;
    
    // Clock generation
    initial begin
        clk = 0;
        forever #5 clk = ~clk;  // 10ns period clock
    end
    
    // Instantiate the design under test
    fsm DUT (
        .clk(clk),
        .rst_n(rst_n),
        .start(start),
        .rw(rw),
        .busy(busy),
        .done(done)
    );
    
    // Test stimulus
    initial begin
        // Dump waves
        $dumpfile("fsm_testbench.vcd");
        $dumpvars(0, fsm_testbench);
        
        $display();
        $display("Starting FSM Test");
        $display("=================");
        $display("Time\tState\t\tInputs\t\tOutputs");
        $display("    \t     \t\trst_n start rw\tbusy done");
        $display("-----------------------------------------------");
        
        // Initialize signals
        rst_n = 0;
        start = 0;
        rw = 0;
        
        // Reset test
        #10;
        $display("%4t\t%s\t%b     %b     %b\t%b    %b", 
                 $time, DUT.current_state.name(), rst_n, start, rw, busy, done);
        
        // Release reset
        rst_n = 1;
        #10;
        $display("%4t\t%s\t%b     %b     %b\t%b    %b", 
                 $time, DUT.current_state.name(), rst_n, start, rw, busy, done);
        
        // Test READ operation
        $display("\n--- Testing READ Operation ---");
        start = 1;
        rw = 0;  // READ
        #10;
        $display("%4t\t%s\t%b     %b     %b\t%b    %b", 
                 $time, DUT.current_state.name(), rst_n, start, rw, busy, done);
        
        start = 0;
        #10;
        $display("%4t\t%s\t%b     %b     %b\t%b    %b", 
                 $time, DUT.current_state.name(), rst_n, start, rw, busy, done);
        
        #10;
        $display("%4t\t%s\t%b     %b     %b\t%b    %b", 
                 $time, DUT.current_state.name(), rst_n, start, rw, busy, done);
        
        #10;
        $display("%4t\t%s\t%b     %b     %b\t%b    %b", 
                 $time, DUT.current_state.name(), rst_n, start, rw, busy, done);
        
        // Test WRITE operation
        $display("\n--- Testing WRITE Operation ---");
        start = 1;
        rw = 1;  // WRITE
        #10;
        $display("%4t\t%s\t%b     %b     %b\t%b    %b", 
                 $time, DUT.current_state.name(), rst_n, start, rw, busy, done);
        
        start = 0;
        #10;
        $display("%4t\t%s\t%b     %b     %b\t%b    %b", 
                 $time, DUT.current_state.name(), rst_n, start, rw, busy, done);
        
        #10;
        $display("%4t\t%s\t%b     %b     %b\t%b    %b", 
                 $time, DUT.current_state.name(), rst_n, start, rw, busy, done);
        
        #10;
        $display("%4t\t%s\t%b     %b     %b\t%b    %b", 
                 $time, DUT.current_state.name(), rst_n, start, rw, busy, done);
        
        // Test staying in IDLE when start is not asserted
        $display("\n--- Testing IDLE Hold ---");
        start = 0;
        rw = 0;
        #20;
        $display("%4t\t%s\t%b     %b     %b\t%b    %b", 
                 $time, DUT.current_state.name(), rst_n, start, rw, busy, done);
        
        // Test reset during operation
        $display("\n--- Testing Reset During Operation ---");
        start = 1;
        rw = 1;
        #10;
        $display("%4t\t%s\t%b     %b     %b\t%b    %b", 
                 $time, DUT.current_state.name(), rst_n, start, rw, busy, done);
        
        // Assert reset
        rst_n = 0;
        #10;
        $display("%4t\t%s\t%b     %b     %b\t%b    %b", 
                 $time, DUT.current_state.name(), rst_n, start, rw, busy, done);
        
        // Release reset
        rst_n = 1;
        start = 0;
        #10;
        $display("%4t\t%s\t%b     %b     %b\t%b    %b", 
                 $time, DUT.current_state.name(), rst_n, start, rw, busy, done);
        
        $display("\n=================");
        $display("Test completed!");
        $display();
        
        #20;
        $finish;
    end

endmodule
```

Verilator Simulation Output:

Starting FSM Test
Time	State		Inputs		Outputs
    	     		rst_n start rw	busy done
-----------------------------------------------
  10	IDLE	0     0     0	0    0
  20	IDLE	1     0     0	0    0

--- Testing READ Operation ---
  30	READ	1     1     0	1    0
  40	DONE	1     0     0	1    1
  50	IDLE	1     0     0	0    0
  60	IDLE	1     0     0	0    0

--- Testing WRITE Operation ---
  70	WRITE	1     1     1	1    0
  80	DONE	1     0     1	1    1
  90	IDLE	1     0     1	0    0
 100	IDLE	1     0     1	0    0

--- Testing IDLE Hold ---
 120	IDLE	1     0     0	0    0

--- Testing Reset During Operation ---
 130	WRITE	1     1     1	1    0
 140	IDLE	0     1     1	0    0
 150	IDLE	1     0     1	0    0

Test completed!

Process finished with return code: 0
Removing Chapter_4_examples/example_5__fsm/obj_dir directory...
Chapter_4_examples/example_5__fsm/obj_dir removed successfully.


0